In [2]:
# type annotations
from typing import List

# generic data handling
import numpy as np

# Music!
import music21
from music21 import *
from music21.stream import Score, Stream

# find+load files
import sys
import glob
import os

## Load Music Files

In [8]:
def load_krns(dataset_dir: str, max_files: int = 5) -> List[Score]:
    """
    Recursively finds all .krn files in a directory. 
    Returns: .krn files converted to music21 Scores
    """
    dataset_dir = os.path.normpath(dataset_dir)
    file_types = ['mid', 'krn']
    paths_partial = []
    for file_type in file_types:
        paths_partial.extend(glob.glob(f'**/**.{file_type}', root_dir=dataset_dir, recursive=True))
    paths_partial = paths_partial[:max_files]
    scores = []
    for path_partial in paths_partial:
        path_full = os.path.join(dataset_dir, path_partial)
        filename = os.path.split(path_full)[-1]

        score = converter.parse(path_full)
        score.metadata.setCustom('filename', filename)
        scores.append(score)
    return scores

In [9]:
scores = load_krns('./datasets')

In [10]:
scores

[<music21.stream.Score 0x2acd0804b30>,
 <music21.stream.Score 0x2acd07a8c80>,
 <music21.stream.Score 0x2acd08075f0>,
 <music21.stream.Score 0x2acd0807710>,
 <music21.stream.Score 0x2acd01cdaf0>]

## Reverse Timing



In [42]:
def reverse_stream(stream: Stream, max_offset: music21.base.OffsetQL = -1) -> Stream:
    """
    Reverse the timing of notes in a stream.
    Parameters:
        max_offset: The offset (in seconds) of the last note. Set manually to synchronize multiple streams. Leave as -1 to auto-compute.
    Returns: A reversed copy of the stream
    """
    new_stream = stream.flatten()
    notes = list(new_stream.notes)
    if max_offset == -1:
        max_offset = max(x.offset for x in notes)
    for n in notes:
        n.offset = max_offset - n.offset
    
    return new_stream

def reverse_score(score: Score) -> Score:
    """
    Reverse the timing of notes in all parts of a score
    Returns: A reversed copy of the score
    """
    new_score = score.cloneEmpty()

    # Determine offset of very last note of all parts
    max_offset = 0.0
    for part in score.parts: 
        part_max_offset = max(x.offset for x in part.flatten().notes)
        max_offset = max(max_offset, part_max_offset)

    # Reverse each part
    for part in score.parts:
        new_score.insert(reverse_stream(part, max_offset))

    return new_score

In [43]:
sample_score = scores[0]

sample_stream = sample_score.parts[0]
sample_stream_reverse = reverse_stream(sample_stream)

print(list(sample_stream.flatten().notes))
print(list(sample_stream_reverse.notes))

sample_stream.write('midi', 'output/reversal/score_part0_orig.mid')
sample_stream_reverse.write('midi', 'output/reversal/score_part0_reverse.mid')


[<music21.note.Note D>, <music21.note.Note D>, <music21.note.Note G>, <music21.note.Note G>, <music21.note.Note F#>, <music21.note.Note G>, <music21.note.Note A>, <music21.note.Note F#>, <music21.note.Note D>, <music21.note.Note C>, <music21.note.Note C>, <music21.note.Note B>, <music21.note.Note C>, <music21.note.Note C>, <music21.note.Note E>, <music21.note.Note D>, <music21.note.Note F#>, <music21.note.Note E>, <music21.note.Note D>, <music21.note.Note C>, <music21.note.Note B>, <music21.note.Note D>, <music21.note.Note E>, <music21.note.Note F#>, <music21.note.Note G>, <music21.note.Note A>, <music21.note.Note B>, <music21.note.Note C>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note E>, <music21.note.Note D>, <music21.note.Note C>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note B>, <music21.note.Note A>, <music21.note.Note D>, <music21.note.Note C#>, <music21.note.Note B>, <music21.note.Note A>, <musi

'output/reversal/score_part0_reverse.mid'

In [44]:
sample_score_reverse = reverse_score(sample_score)

sample_score.write('midi', 'output/reversal/score_full_orig.mid')
sample_score_reverse.write('midi', 'output/reversal/score_full_reverse.mid')

'output/reversal/score_full_reverse.mid'

(<music21.metadata.primitives.Text xyz file>,)

0.0

## Save Scores to .midi



In [7]:
scores[0].write('midi', 'output/test/test_score_0.mid')

'output/test/test_score_0.mid'

## Load 